In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'
import sys; print(sys.version)

3.8.2 (default, Mar 26 2020, 15:53:00) 
[GCC 7.3.0]


In [2]:
from import_all import *

In [3]:
def hang_path(t_in,t_out,rename_base):
    return lambda fn : t_out/rename_base(t_in)/Path(os.path.relpath(str(fn),str(t_in)))

# Preprocesamiento de imágenes

In [5]:
#IN_PATH = Path('./data/cedar')
IN_PATH = Path('./data/sigcomp2009')

OUT_PATH = Path('./data/parodi_out/preprocessed')

rename_base = lambda x : x.name

#get_fns = get_image_files
get_fns = ds_sigcomp2009.get_authentic_signatures


fns = L(get_fns(IN_PATH))
nf_func = hang_path(IN_PATH,OUT_PATH,rename_base)
def preproc_fmap(fn):
    nf = nf_func(fn)
    nf.parent.mkdir(parents=True, exist_ok=True)
    return otsu_preprocess(ds_sigcomp2009.get_x(fn)).save(nf)
preproc_fns = parallel(preproc_fmap,fns)

# Data augmentation

In [4]:
IN_PATH = Path('./data/parodi_out/preprocessed/sigcomp2009')
OUT_PATH = Path('./data/parodi_out/preprocessed/augmented')
get_fns = ds_sigcomp2009.get_authentic_signatures
rename_base = lambda x : x.name

fns = L(get_fns(IN_PATH))
nf_func = hang_path(IN_PATH,OUT_PATH,rename_base)
def augment_fmap(fn):
    new_parent = nf_func(fn).parent
    new_parent.mkdir(parents=True, exist_ok=True)
    augmented_imgs = data_augmentation(np.array(ds_sigcomp2009.get_x(fn)), 10)
    res = []
    for i, nimg in enumerate(augmented_imgs):
        new_filename = str(fn.name)[:-7] + str(i) + '_6g.PNG'
        new_path = new_parent/Path(new_filename) 
        nimg.save(new_path)
        res.append(new_path)
    return res
aug_fns = parallel(augment_fmap,fns)


# Extracción de features con grilla circula

In [41]:
%%time
IN_PATH = Path('./data/parodi_out/preprocessed/augmented-sigcomp2009')
ds_name = IN_PATH.name

for NDIV in [8,16,32,64,128]:
    print(NDIV)
    get_fns = get_image_files
    fns = L(get_fns(IN_PATH))
    def extract_fmap(fn):
        g = FastGrid(fn, lambda f : PILImageBW.create(f), N_divisions=NDIV)
        return g.features()

    OUT_PATH = Path('./data/parodi_out/features')/Path(str(NDIV))
    OUT_PATH.mkdir(parents=True, exist_ok=True)

    extracted = parallel(extract_fmap,fns)
    with open(OUT_PATH/Path(ds_name+'.npy'), 'wb') as f:
        np.save(f,np.array(extracted))
    with open(OUT_PATH/Path(ds_name+'.txt'), 'w') as f:
        f.write('\n'.join(fns.map(str)))

8
16
32
64
128
CPU times: user 40.5 s, sys: 5.03 s, total: 45.5 s
Wall time: 2h 46min 21s


In [4]:
for ndiv in [8,16,32,64,128]:
    print(ndiv)
    for ds_name in ['augmented-sigcomp2009', 'sigcomp2009', 'cedar']:
        print(ds_name)
        fns,x=load_features(ds_name,ndiv)
        print(len(fns),x.shape)

8
augmented-sigcomp2009
10000 (10000, 3, 8)
sigcomp2009
1000 (1000, 3, 8)
cedar
1320 (1320, 3, 8)
16
augmented-sigcomp2009
10000 (10000, 3, 16)
sigcomp2009
1000 (1000, 3, 16)
cedar
1320 (1320, 3, 16)
32
augmented-sigcomp2009
10000 (10000, 3, 32)
sigcomp2009
1000 (1000, 3, 32)
cedar
1320 (1320, 3, 32)
64
augmented-sigcomp2009
10000 (10000, 3, 64)
sigcomp2009
1000 (1000, 3, 64)
cedar
1320 (1320, 3, 64)
128
augmented-sigcomp2009
10000 (10000, 3, 128)
sigcomp2009
1000 (1000, 3, 128)
cedar
1320 (1320, 3, 128)
